### Knock Out 

In [1]:
import cobra
model = cobra.io.read_sbml_model('/data1/xpgeng/cross_pathogen/GEM_model/iML1515.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
import csv
one_gene = {}
with open('/data1/xpgeng/cross_pathogen/one_two_knockout/' +'iML1515_1KO_growth', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) 
    for row in reader:
        #print(len(row))
        if len(row) == 4:            
            one_gene[row[0]] = int(row[3])           
    print(len(one_gene)) 

1516


In [3]:
gene_list = []
for k,v in one_gene.items():
    gene_list.append(k)
gene_list.remove('s0001')
gene_list = sorted(gene_list)
print(len(gene_list))

1515


In [4]:
gene_dic = {}
for i in range(1515):
    gene_dic[gene_list[i]] = i

In [5]:
import csv
from itertools import combinations
from tqdm import tqdm
import cobra.flux_analysis
from multiprocessing import Pool

# Adjust the function to use model and gene_list as arguments
def knockout(gene, gene_list, model):
    
    rest_list = gene_list[gene_dic[gene]+1:]
    twogene = list(combinations(rest_list, 2))
    
    filepath = f'iML1515_{gene}.csv'

    with open('/data1/xpgeng/cross_pathogen/FBA/iML1515-3KO/' + filepath, mode='a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Gene', 'Gene1', 'Gene2', 'Initial Growth', 'Altered Growth', 'Label'])

        initial_state = cobra.flux_analysis.pfba(model)
        initial_growth = round(initial_state.fluxes['BIOMASS_Ec_iML1515_core_75p37M'], 3)

        for gene1, gene2 in tqdm(twogene):
            with model:
                try:
                    model.genes.get_by_id(gene).knock_out()
                    model.genes.get_by_id(gene1).knock_out()
                    model.genes.get_by_id(gene2).knock_out()
                    altered_state = cobra.flux_analysis.pfba(model)
                    altered_growth = round(altered_state.fluxes['BIOMASS_Ec_iML1515_core_75p37M'], 3)
                    label = 1 if altered_growth < 0.1 * initial_growth else 0
                    writer.writerow([gene, gene1, gene2, initial_growth, altered_growth, label])
                except Exception as e:
                    writer.writerow([gene, gene1, gene2, 'Failed', str(e)])

In [ ]:
def run_knockouts(gene_list, model):
    # Slice the first 100 genes from the list
    top_genes = gene_list[1000:]
    
    # Create a multiprocessing pool with 100 workers
    with Pool(100) as pool:
        # Each pool worker will execute the `knockout` function with a gene, the full gene list, and the model
        results = pool.starmap(knockout, [(gene, gene_list, model) for gene in top_genes])

if __name__ == "__main__":
    run_knockouts(gene_list, model)

  0%|                                       | 1/77421 [00:00<8:09:41,  2.63it/s]

### Merge All Data 

In [9]:
# combine all data
from tqdm import tqdm
import os
import csv

# 文件夹路径
folder_path = '/data1/xpgeng/cross_pathogen/FBA/iML1515-3KO/'

# 输出文件路径
output_file = os.path.join('/data1/xpgeng/cross_pathogen/FBA/', 'iML1515_all.csv')

filter_keywords = ['Fail', 'b2092', 'b4104']

# 打开输出文件准备写入
with open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    
    # 不写入header
    #writer.writerow(['gene1', 'gene2', 'gene3', 'label'])

    # 遍历文件夹中的所有 CSV 文件
    for filename in tqdm(os.listdir(folder_path)):
        if filename.endswith('.csv'):  # 筛选 .csv 文件
            file_path = os.path.join(folder_path, filename)
            
            # 读取文件并逐行处理
            with open(file_path, mode='r', encoding='utf-8') as infile:
                reader = csv.reader(infile)
                for i, row in enumerate(reader):
                    # 跳过原始文件的表头
                    if i == 0:
                        continue
                    
                    # 如果该行不包含 'Fail', 'b2092', 'b4104'，写入到输出文件
                    if not any(keyword in str(cell) for cell in row for keyword in filter_keywords):
                        writer.writerow([row[0], row[1], row[2], row[5]])
                #break

print(f"ok saved in {output_file}")

100%|███████████████████████████████████████| 1516/1516 [35:05<00:00,  1.39s/it]

ok saved in /data1/xpgeng/cross_pathogen/FBA/iML1515_all.csv


In [10]:
import pandas as pd

# 文件路径
file_path = '/data1/xpgeng/cross_pathogen/FBA/iML1515_all.csv'

# 读取 CSV 文件
df = pd.read_csv(file_path, header=None)  # 如果没有表头，使用 header=None

# 输出行数
num_rows = len(df)
print(f"Total rows in the file '{file_path}': {num_rows}")

Total rows in the file '/data1/xpgeng/cross_pathogen/FBA/iML1515_all.csv': 576107619


C(1513,3) = 576,108,036 \
417 data missing.

### Split the data into 10 parts 

In [1]:
import pandas as pd
import os

# 读取最终合并的 CSV 文件
input_file = '/data1/xpgeng/cross_pathogen/FBA/iML1515_all.csv'
output_folder = '/data1/xpgeng/cross_pathogen/FBA/iML1515_parts/'

# 读取 CSV 文件，假设没有表头
df = pd.read_csv(input_file, header=None)

# 打乱数据
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 计算每份数据的大小
num_splits = 10
split_size = len(df) // num_splits

# 划分数据并保存
for i in range(num_splits):
    # 为每一份计算开始和结束的索引
    start_idx = i * split_size
    # 最后一份可能包含剩余所有数据
    end_idx = (i + 1) * split_size if i < num_splits - 1 else len(df)
    
    # 获取数据
    df_split = df.iloc[start_idx:end_idx]

    # 输出文件名
    output_file = os.path.join(output_folder, f'iML1515-{i+1}.csv')
    
    # 保存数据到文件
    df_split.to_csv(output_file, index=False, header=False)

print(f"Data successfully split into {num_splits} parts and saved in {output_folder}")

Data successfully split into 10 parts and saved in /data1/xpgeng/cross_pathogen/FBA/iML1515_parts/


In [2]:
import pandas as pd
import os

# 文件夹路径
folder_path = '/data1/xpgeng/cross_pathogen/FBA/iML1515_parts/'

# 初始化行数总和
total_rows = 0

# 遍历文件夹中的所有 CSV 文件
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # 筛选 .csv 文件
        file_path = os.path.join(folder_path, filename)
        
        # 读取 CSV 文件
        df = pd.read_csv(file_path, header=None)  # 如果没有表头，使用 header=None
        
        # 计算当前文件的行数
        num_rows = len(df)
        
        # 更新总行数
        total_rows += num_rows
        
        # 输出文件名和前 3 行数据
        print(f"First 3 rows of {filename}:")
        print(df.head(3))  # 显示前 3 行
        print(f"Total rows in {filename}: {num_rows}")
        print("\n" + "="*50 + "\n")

# 输出所有文件的总行数
print(f"Total rows across all files: {total_rows}")

First 3 rows of iML1515-1.csv:
       0      1      2  3
0  b2425  b2484  b3942  0
1  b0085  b3744  b4209  1
2  b1524  b3008  b4407  1
Total rows in iML1515-1.csv: 57610761


First 3 rows of iML1515-2.csv:
       0      1      2  3
0  b2540  b2838  b3256  1
1  b2507  b3374  b3727  1
2  b2788  b3607  b4254  1
Total rows in iML1515-2.csv: 57610761


First 3 rows of iML1515-3.csv:
       0      1      2  3
0  b0315  b1587  b3880  0
1  b2312  b3089  b3909  1
2  b2053  b3177  b3784  1
Total rows in iML1515-3.csv: 57610761


First 3 rows of iML1515-4.csv:
       0      1      2  3
0  b0125  b0158  b1033  0
1  b1101  b3656  b4006  1
2  b1019  b1812  b4019  1
Total rows in iML1515-4.csv: 57610761


First 3 rows of iML1515-5.csv:
       0      1      2  3
0  b0789  b3128  b4077  0
1  b0040  b0315  b2097  0
2  b0325  b1054  b2173  0
Total rows in iML1515-5.csv: 57610761


First 3 rows of iML1515-6.csv:
       0      1      2  3
0  b0734  b2284  b3952  0
1  b0149  b2670  b3478  0
2  b0733  b1638 

In [4]:
from itertools import combinations
twogene=list(combinations(gene_list,2))
from tqdm import tqdm

In [6]:
with open('iML1515_2KO_growth', mode='a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    firstrow = ['gene, label']
    writer.writerow(firstrow)  

    state_0   = cobra.flux_analysis.pfba(model)      
    growth_0    = round(state_0['BIOMASS_Ec_iML1515_core_75p37M'], 3)

    for i in tqdm(twogene):   
        with model:                
            try:            
                model.genes.get_by_id(i[0]).knock_out()
                model.genes.get_by_id(i[1]).knock_out()
                state_1   = cobra.flux_analysis.pfba(model)
                growth_1    = round(state_1['BIOMASS_Ec_iML1515_core_75p37M'], 3)
                if growth_1 < 0.5*growth_0:
                    
                    writer.writerow([i[0], i[1], growth_0, growth_1, 1]) 
                else:
                    writer.writerow([i[0], i[1], growth_0, growth_1, 0])
            except:
                writer.writerow([i, 'fail'])

100%|████████████████████████████████| 860016/860016 [63:12:54<00:00,  3.78it/s]


In [5]:
two_gene = {}
with open('/data1/xpgeng/cross_pathogen/FBA/' +'iML1515_2KO_growth', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) 
    for row in reader:
        #print(len(row))
        if len(row) == 5:            
            two_gene[row[0], row[1]] = int(row[4])           
    print(len(two_gene)) 

860012


In [6]:
two_list = []
for k,v in two_gene.items():
    if v == 1:
        two_list.append(k[0])
        two_list.append(k[1])
print(two_list)

['b2551', 'b0907', 'b2551', 'b4388', 'b2551', 'b2913', 'b0870', 'b0907', 'b0870', 'b2320', 'b2436', 'b3867', 'b3500', 'b0888', 'b2465', 'b2935', 'b2836', 'b0954', 'b4054', 'b0928', 'b3281', 'b1692', 'b0844', 'b3812', 'b0126', 'b0339', 'b2508', 'b0238', 'b2508', 'b2868', 'b2508', 'b2867', 'b2508', 'b2866', 'b3846', 'b0954', 'b0474', 'b4383', 'b0474', 'b1207', 'b0474', 'b4094', 'b3008', 'b1622', 'b1190', 'b4053', 'b0910', 'b0171', 'b3926', 'b3608', 'b4383', 'b1207', 'b2708', 'b3197', 'b0077', 'b3671', 'b0077', 'b3670', 'b0078', 'b3671', 'b0078', 'b3670', 'b1849', 'b2500', 'b0099', 'b1865', 'b3386', 'b2029', 'b0073', 'b1800', 'b2421', 'b2414', 'b0002', 'b3940', 'b1096', 'b4039', 'b3117', 'b3772', 'b0273', 'b4254', 'b3744', 'b0674', 'b3608', 'b1727', 'b1207', 'b4094', 'b2422', 'b0763', 'b2422', 'b0764', 'b2422', 'b0765', 'b0763', 'b2423', 'b0763', 'b2424', 'b0764', 'b2423', 'b0764', 'b2424', 'b2423', 'b0765', 'b0765', 'b2424', 'b3213', 'b1761', 'b1761', 'b3212', 'b0452', 'b0954', 'b0048', 

In [24]:
print(len(set(two_list)))

108
